# `AsyncLocal<T>` and the TPL
This project demonstrates how the behavior of [`AsyncLocal<T>`](https://docs.microsoft.com/en-us/dotnet/api/system.threading.asynclocal-1) can vary depending upon how tasks are created.

In [ ]:
using System.Threading;
using System.Threading.Tasks;

static AsyncLocal<int> myLocal = new AsyncLocal<int>();
myLocal.Value = 1;

static async Task Increment(int count){   
    int prevValue = myLocal.Value;
    Console.WriteLine("Before: Previous: {0}, Current: {1}", prevValue, myLocal.Value);
    await Task.Delay(1);
    myLocal.Value = prevValue + 1;
    Console.WriteLine("After: Previous: {0}, Current: {1}", prevValue, myLocal.Value);

    if(myLocal.Value < count)
        await Increment(count);
}

// call 10 recursive awaits
await Increment(10);

// final
Console.WriteLine("Final: {0}", myLocal.Value);

Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 2, Current: 2
After: Previous: 2, Current: 3
Before: Previous: 3, Current: 3
After: Previous: 3, Current: 4
Before: Previous: 4, Current: 4
After: Previous: 4, Current: 5
Before: Previous: 5, Current: 5
After: Previous: 5, Current: 6
Before: Previous: 6, Current: 6
After: Previous: 6, Current: 7
Before: Previous: 7, Current: 7
After: Previous: 7, Current: 8
Before: Previous: 8, Current: 8
After: Previous: 8, Current: 9
Before: Previous: 9, Current: 9
After: Previous: 9, Current: 10
Final: 1


In the above instance, the `AsyncLocal<T>` value is recursively preserved all the way down the `await` chain until it reaches a value of 10. However, notice that the final value of the `AsyncLocal<T>` is `1`. Let's explore that.

In [ ]:
static AsyncLocal<int> myLocal2 = new AsyncLocal<int>();
myLocal2.Value = 1;

static async Task Increment2(){   
    int prevValue = myLocal2.Value;
    Console.WriteLine("Before: Previous: {0}, Current: {1}", prevValue, myLocal2.Value);
    await Task.Delay(1);
    myLocal2.Value = prevValue + 1;
    Console.WriteLine("After: Previous: {0}, Current: {1}", prevValue, myLocal2.Value);
}

// call 10 awaits that all return to the main caller thread
foreach(var i in Enumerable.Range(0, 10))
    await Increment2();

Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2


In the second example, where the `await` calls all return to the context of the original caller, the `AsyncLocal<T>` returns back to its original value each time. `AsyncLocal<T>` will preserve the value that began at the start of this context and will propgate changes down the `await` FSM, but those side effects aren't propagated back to the original caller. `AsyncLocal<T>` value changes can be carried down the `await` FSM but they're not returned to the original caller - that's what `Task<TValue>` return values are for.

## `AsyncLocal<T>` and `ContinueWith<T>`
So how does `AsyncLocal<T>` work with continuation tasks?

In [ ]:
static AsyncLocal<int> myLocal3 = new AsyncLocal<int>();
myLocal3.Value = 1;

static void LogAndIncrement(){
    int prevValue = myLocal3.Value;
    Console.WriteLine("Before: Previous: {0}, Current: {1}", prevValue, myLocal3.Value);
    myLocal3.Value = prevValue + 1;
    Console.WriteLine("After: Previous: {0}, Current: {1}", prevValue, myLocal3.Value);
}

Task t1 = Task.Run(() => {
    LogAndIncrement();
});

Task t2 = t1.ContinueWith(tr => {
    LogAndIncrement();
});

await t2;

Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2


Again, a similar result to when we ran `await` in a `foreach` loop - values that are modified outside of the `await` FSM aren't propagated back to the caller. `AsyncLocal<T>` works in one direction: down the FSM.

In [ ]:
static AsyncLocal<int> myLocal4 = new AsyncLocal<int>();
myLocal4.Value = 1;

static async Task LogAndIncrement2(){
    int prevValue = myLocal4.Value;
    Console.WriteLine("Before: Previous: {0}, Current: {1}", prevValue, myLocal4.Value);
    await Task.Delay(1);
    myLocal4.Value = prevValue + 1;
    Console.WriteLine("After: Previous: {0}, Current: {1}", prevValue, myLocal4.Value);
}


Task t3 = Task.Run(async () => {
    await LogAndIncrement2();
    await LogAndIncrement2();
});

await t3;

Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2


Again, the `await`s that occur inside the `Task` are part of separate FSMs, thus we always return back to the original value we began with at the start of this `async` context.

In [ ]:
static AsyncLocal<int> myLocal5 = new AsyncLocal<int>();
myLocal5.Value = 1;

Task t4 = Task.Run(async () => {
    foreach(var i in Enumerable.Range(0, 10)){
        int prevValue = myLocal5.Value;
        Console.WriteLine("Before: Previous: {0}, Current: {1}", prevValue, myLocal5.Value);
        await Task.Delay(1);
        prevValue = myLocal5.Value;
        myLocal5.Value = prevValue + 1;
        Console.WriteLine("After: Previous: {0}, Current: {1}", prevValue, myLocal5.Value);
    }
});

await t4;

Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 2, Current: 2
After: Previous: 2, Current: 3
Before: Previous: 3, Current: 3
After: Previous: 3, Current: 4
Before: Previous: 4, Current: 4
After: Previous: 4, Current: 5
Before: Previous: 5, Current: 5
After: Previous: 5, Current: 6
Before: Previous: 6, Current: 6
After: Previous: 6, Current: 7
Before: Previous: 7, Current: 7
After: Previous: 7, Current: 8
Before: Previous: 8, Current: 8
After: Previous: 8, Current: 9
Before: Previous: 9, Current: 9
After: Previous: 9, Current: 10
Before: Previous: 10, Current: 10
After: Previous: 10, Current: 11


This is probably much closer to what users expect from `AsyncLocal<T>` - within the context of a single `async` method running inside a `Task` we have 10 discrete `await` calls, each of which creates its own stage in the `await` FSM and properly increments the value stored inside `myLocal5`.

But what happens when we tack on a `ContinueWith` here?

In [ ]:
static AsyncLocal<int> myLocal6 = new AsyncLocal<int>();
myLocal6.Value = 1;

// force the foreground thread to wait until continuation has executed
ManualResetEvent m = new ManualResetEvent(false);

Task t5 = Task.Run(async () => {
    foreach(var i in Enumerable.Range(0, 10)){
        int prevValue = myLocal6.Value;
        Console.WriteLine("Before: Previous: {0}, Current: {1}", prevValue, myLocal6.Value);
        await Task.Delay(1);
        prevValue = myLocal6.Value;
        myLocal6.Value = prevValue + 1;
        Console.WriteLine("After: Previous: {0}, Current: {1}", prevValue, myLocal6.Value);
    }
}).ContinueWith(async tr => {
    await Task.Delay(1);
    int prev2 = myLocal6.Value;
    Console.WriteLine("Final Previous: {0}, Current: {1}", prev2, myLocal6.Value);
    m.Set(); 
});

await t5;
m.WaitOne();

Before: Previous: 1, Current: 1
After: Previous: 1, Current: 2
Before: Previous: 2, Current: 2
After: Previous: 2, Current: 3
Before: Previous: 3, Current: 3
After: Previous: 3, Current: 4
Before: Previous: 4, Current: 4
After: Previous: 4, Current: 5
Before: Previous: 5, Current: 5
After: Previous: 5, Current: 6
Before: Previous: 6, Current: 6
After: Previous: 6, Current: 7
Before: Previous: 7, Current: 7
After: Previous: 7, Current: 8
Before: Previous: 8, Current: 8
After: Previous: 8, Current: 9
Before: Previous: 9, Current: 9
After: Previous: 9, Current: 10
Before: Previous: 10, Current: 10
After: Previous: 10, Current: 11
Final Previous: 1, Current: 1


As you can see, the final value for `myLocal6` reverts back to 1 - as the `async` flow / the `await` FSM are separate, unrelated instances between the parent `Task` and the continuation `Task`, even though users very likely view them as one and the same.